## Extract a JSON events sidecar template from a BIDS dataset.

General strategy for machine-actionable annotation using HED in BIDS is
to create a single `events.json` sidecar file in the BIDS dataset root directory.
Ideally, this sidecar will contain all the annotations needed for users to
understand and analyze the data.
(See the [**BIDS annotation quickstart**](https://hed-examples.readthedocs.io/en/latest/BidsAnnotationQuickstart.html) for additional
information on this strategy.)

This notebook shows how to create a JSON sidecar template from the information in all
the event files in a BIDS dataset.

To use this notebook, substitute the specifics of your BIDS
dataset for the following variables:

| Variable | Purpose |
| -------- | ------- |
| bids_root_path | Full path to root directory of dataset.|
| exclude_dirs | List of directories to exclude when constructing the list of event files. |
| name_indices  | Indices used to construct a unique keys representing event filenames.<br>(See [Dictionaries of filenames](dictionaries-of-filenames-anchor) for examples of how to choose these indices.)|
| skip_columns  |  List of column names in the `events.tsv` files to skip in the analysis. |
| value_columns | List of columns names in the `events.tsv` files to annotate as<br>as a whole rather than by individual column value. |


For large datasets, be sure to exclude columns such as
`onset` and `sample`, since the summary produces counts of the number of times
each unique value appears somewhere in dataset event files.

When run, the script creates a dictionary of the unique values in each column
by consolidating the information in all of the `events.tsv` files in the dataset.
It then outputs the result as a JSON string representing a JSON sidecar.

The example below uses a
[small version](https://github.com/hed-standard/hed-examples/tree/main/datasets/eeg_ds003654s_hed)
of the Wakeman-Hanson face-processing dataset available on openNeuro as
[ds003654](https://openneuro.org/datasets/ds003645/versions/2.0.0).


In [1]:
import os
import json
from hed.tools import EventFileDictionary, EventValueSummary
from hed.util import get_file_list

# Variables to set for the specific dataset
bids_root_path =  os.path.realpath('../../../datasets/eeg_ds003654s_hed')
exclude_dirs = ['stimuli']
name_indices = (0, 2)
skip_columns = ["onset", "duration", "sample", "trial", "response_time"]
value_columns = ["rep_lag", "stim_file", "value"]

# Construct the event file dictionary for the BIDS event files
event_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)
file_dict = EventFileDictionary(event_files, name_indices=name_indices)

# Construct the event file value summary and extract a sidecar template representing dataset
value_summary = EventValueSummary(value_cols=value_columns, skip_cols=skip_columns, name="Wakeman-Hanson test data")
value_summary.update(event_files)
sidecar_template = value_summary.extract_sidecar_template()
str_json = json.dumps(sidecar_template, indent=4)
print(str_json)


{
    "event_type": {
        "Description": "Description for event_type",
        "HED": {
            "double_press": "Label/double_press",
            "left_press": "Label/left_press",
            "right_press": "Label/right_press",
            "setup_right_sym": "Label/setup_right_sym",
            "show_circle": "Label/show_circle",
            "show_cross": "Label/show_cross",
            "show_face": "Label/show_face",
            "show_face_initial": "Label/show_face_initial"
        },
        "Levels": {
            "double_press": "Description for double_press",
            "left_press": "Description for left_press",
            "right_press": "Description for right_press",
            "setup_right_sym": "Description for setup_right_sym",
            "show_circle": "Description for show_circle",
            "show_cross": "Description for show_cross",
            "show_face": "Description for show_face",
            "show_face_initial": "Description for show_face_initial"
   